In [1]:
import socket
import pickle
from amoba import gomoku

In [2]:
G=gomoku()
G.fig

Figure(axes=[Axis(grid_color='black', scale=LinearScale(max=15.5, min=-1.5), tick_style={'font-size': 0}, tick…

In [3]:
G.reset_table()
# Create a TCP/IP socket
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# Bind the socket to the port
server_address = ('localhost', 10000)
sock.bind(server_address)
# Listen for incoming connections
sock.listen(1)

In [4]:
while not(G.check_win()):
    # Wait for a connection
    connection, client_address = sock.accept()
    try:
        # Receive the data in small chunks and retransmit it
        while True:
            data = connection.recv(8192)
            if data:
                data = pickle.loads(data)
                succesful_update = G.update_table(pos = data['pos'],player=data['player'])                   
                connection.sendall(pickle.dumps(dict(A=G.A,
                                                     who_is_next=G.who_is_next,
                                                     end_of_game=G.check_win(),
                                                     succesful_update = succesful_update)))
            else:
                break
            
    except (KeyboardInterrupt, SystemExit):
        
        raise
connection.close()
sock.close()